## DIA 034: Implementación de un Pipeline de Retraining Basado en Feedback

En el Día 35, se ha añadido una funcionalidad para actualizar el modelo de predicción de manera dinámica utilizando los datos de feedback recolectados. La idea es que, a medida que los usuarios envían comentarios sobre la exactitud de las predicciones, esta información pueda utilizarse para reentrenar y mejorar el modelo. Para ello, se ha implementado un endpoint para administradores que inicia el proceso de retraining de forma asíncrona (simulado aquí con un proceso en segundo plano). Este proceso se ejecuta de manera separada sin bloquear la respuesta de la API, y al finalizar, se actualiza el modelo (en este ejemplo, se simula la actualización guardando un archivo).

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import os
import logging
from datetime import datetime
import time
import threading
import json

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos (simplificados para este ejemplo)
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para simplificar

# Modelo Feedback (definido en días anteriores, omitido aquí para centrarnos en retraining)

# ---------------------------
# Funciones de Decoradores (por ejemplo, role_required, si se usa en endpoints administrativos)
# ---------------------------
from functools import wraps
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            # Para este ejemplo, se asume que el campo 'role' existe y se compara con el string requerido.
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Endpoint de Login (común)
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username', None)
    password = data.get('password', None)
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Aquí se asume que la validación es exitosa (en un caso real, se verificarían las credenciales)
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

# ---------------------------
# Endpoints de Versionado (v1 y v2, definidos en días anteriores)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    # Simulación simple: siempre retorna dígito 5 con 90% de probabilidad
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1"}
    logger.info("v1: Predicción realizada.")
    return jsonify(result), 200

api_v2 = Blueprint('api_v2', __name__)
@api_v2.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("150 per day")
def predict_v2():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400
    result = {"prediccion": 7, "probabilidad": 0.95, "version": "v2", "mensaje": "Predicción mejorada"}
    logger.info("v2: Predicción realizada con mejoras.")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')
app.register_blueprint(api_v2, url_prefix='/api/v2')

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Nuevo Endpoint: Pipeline de Retraining Basado en Feedback
# ---------------------------
@app.route('/admin/retrain', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_model_endpoint():
    """
    Endpoint para que un administrador inicie el proceso de retraining del modelo basado en feedback.
    Este proceso se ejecuta de manera asíncrona para no bloquear la respuesta de la API.
    """
    def retrain_job():
        logger.info("Inicio del retraining del modelo basado en feedback...")
        # Simulación del proceso de retraining (puedes reemplazar esto con tu lógica de retraining real)
        time.sleep(10)  # Simula el tiempo que tarda el retraining
        # Simula la actualización del modelo (por ejemplo, guarda un nuevo modelo en un archivo)
        with open('updated_model.h5', 'w') as f:
            f.write("Modelo actualizado basado en feedback")
        logger.info("Retraining completado. Modelo actualizado.")

    # Ejecutar el proceso de retraining en un hilo separado
    thread = threading.Thread(target=retrain_job)
    thread.start()
    return jsonify({"msg": "Proceso de retraining iniciado"}), 202

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Configuración Inicial y Extensiones:

Se configura la aplicación Flask, las extensiones (JWT, SQLAlchemy, Limiter, etc.) y el logging.
Se definen endpoints comunes (login, health) y los endpoints versionados para /predict usando Blueprints.
Nuevo Endpoint de Retraining:

Se añade el endpoint /admin/retrain protegido con @jwt_required() y @role_required('admin') para que solo los administradores puedan iniciar el proceso.
La función retrain_job() simula un proceso de retraining del modelo (por ejemplo, espera 10 segundos y escribe un archivo simulado "updated_model.h5").
El proceso de retraining se ejecuta en un hilo separado para no bloquear la respuesta inmediata al cliente.
Registro de Blueprints:

Se registran los Blueprints para versiones v1 y v2 con sus respectivos prefijos, permitiendo evolucionar la API sin afectar a los clientes existentes.
Ejecutar la Aplicación:

La aplicación se inicia en modo debug.